<a href="https://colab.research.google.com/github/MustiCankan/SpiriteAI/blob/main/SameNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OMNIGEN UI Gradio Same v1.2.1

In [ ]:
!pip install gradio
!pip install git+https://github.com/huggingface/diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-edgptk5v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-edgptk5v
  Resolved https://github.com/huggingface/diffusers to commit 924f880d4da3af3d376ed8d834e613192071fee4
  Installing build dependencies ... done
  Getti

In [ ]:
import gradio as gr
from diffusers import DiffusionPipeline, FluxPipeline
import torch
from diffusers import OmniGenPipeline
from diffusers.utils import load_image


pipe = OmniGenPipeline.from_pretrained(
    "Shitao/OmniGen-v1-diffusers",
    torch_dtype=torch.bfloat16
)
pipe.to("cuda")


def load_model(input_selected_model):
    return input_selected_model
def show_image(img):
    print(img)
    return img

def generate_image_with_progress(prompt, step_size, guidance_scale, input_image1, input_image2,image_guidance_scale,seed):
    step_size = int(step_size)
    guidance_scale = float(guidance_scale)

    input_images = []

    if input_image1 != None:
        input_images.append(load_image(input_image1))
        print("Image 1 is loaded.")
    if input_image2 != None:
        input_images.append(load_image(input_image2))
        print("Image 2 is loaded.")

    #  Check condition if input_images are avaliable
    if len(input_images) > 0:
      print("There are images,so pipeline modified.")
      image = pipe(
          prompt=prompt,
          input_images=input_images,
          height=1024,
          width=1024,
          num_inference_steps=step_size,
          guidance_scale=guidance_scale,
          img_guidance_scale=image_guidance_scale,
          generator=torch.Generator(device="cpu").manual_seed(seed)
          ).images[0]
    else:
    # Running the pipeline
        image = pipe(
          prompt=prompt,
          height=1024,
          width=1024,
          num_inference_steps=step_size,
          guidance_scale=guidance_scale,
          img_guidance_scale=image_guidance_scale,
          generator=torch.Generator(device="cpu").manual_seed(seed)
          ).images[0]

    image.save("output.png")

    return image

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Group():
            gr.Markdown("### Spirit AI Model Extractor (SAME) v1.2.1", elem_id="name_l")

            logo = gr.Image(value="/content/logo2.png", show_label=False, elem_id="logo-image")

            with gr.Accordion("Upload Images", open=False):
              with gr.Column():
                input_image1 =gr.Image(label="Input Image Component 1", type="filepath")
                input_image2 =gr.Image(label="Input Image Component 2", type="filepath")
        with gr.Column():

            prompt = gr.Textbox(label="Prompt")
            with gr.Accordion("Advanced options", open=False):
                step_size = gr.Number(label="Step Size", value=50)
                guidance_scale = gr.Number(label="Guidance Scale", value=3)
                image_guidance_scale = gr.Number(label="Image Guidance Scale", value=1.6)
                seed = gr.Number(label="Seed", value=42)
            generate_image_btn = gr.Button(value="Generate Image")
            show_image_btn = gr.Button(value="Show Image")
            examples = gr.Examples(
                examples=["Blonde hair girl", "Bald hair girl",
                          "A woman is holding and prominently showcasing a canned cat food. The woman has [describe appearance if needed]. The reference for the canned cat food is '[input]'. Ensure the can is clearly visible and detailed, with branding and design intact.",
                          "Realistic photo. A young woman sits on a sofa, holding a book and facing the camera. She wears delicate silver hoop earrings adorned with tiny, sparkling diamonds that catch the light, with her long chestnut hair cascading over her shoulders. Her eyes are focused and gentle, framed by long, dark lashes. She is dressed in a cozy cream sweater, which complements her warm, inviting smile. Behind her, there is a table with a cup of water in a sleek, minimalist blue mug. The background is a serene indoor setting with soft natural light filtering through a window, adorned with tasteful art and flowers, creating a cozy and peaceful ambiance. 4K, HD."
                          ],
                inputs=[prompt]
            )
        with gr.Column():
            output_image = gr.Image(label="Generated Image")


        # Custom CSS for resizing the logo
        demo.css = """
    #logo-image img {
      width: 500px;
      height: auto;
      display: block;
      margin: 5px auto 20px auto; /* bottom margin of 20px */
      padding: 8px;
      }

  #name_l  {
    max-width: 100%;
    height: auto;
    display: block;
    margin: 5 auto;
    padding: 8;
    }
    """


    generate_image_btn.click(
        fn=generate_image_with_progress,
        inputs=[prompt, step_size, guidance_scale, input_image1, input_image2,image_guidance_scale,seed],
        outputs=output_image
    )
    show_image_btn.click(
        fn=show_image,
        inputs=input_image1,
        outputs=output_image
    )


demo.launch(share=True, debug=True)


Loading pipeline components...:   0%|          | 0/4 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7e61e63fca5a8dcc4a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
